# Introduction and setup

In this notebook, I have code to load in the stellar profiles in 3 dimensions which 
were previously calculated on the command line so that we can plot quantities such as 
the 1D velocity dispersion and evolution with time of the mass.

In [ ]:
using Revise
using LilGuys
using CairoMakie, Arya

In [ ]:
modelsdir = ENV["DWARFS_ROOT"] * "/analysis"

In [ ]:
import TOML
obs_props = TOML.parsefile(joinpath(ENV["DWARFS_ROOT"], "observations/sculptor/observed_properties.toml"))

In [ ]:
function load_profile(modelname, starsname)
    orbit = load_profiles(modelname * "/stars/" * starsname)

    return orbit
end

In [ ]:
function load_isolation(modelname, starsname)
    iso = load_profiles(modelname * "/../stars/" * starsname)
    return iso
end

In [ ]:
function load_profiles(filename; extension="stellar_profiles_3d.hdf5")
    path = joinpath(modelsdir, filename, extension)
    
    profs = LilGuys.read_structs_from_hdf5(path, LilGuys.StellarProfile3D)
    idxs = parse.(Int, first.(profs))
    profs = last.(profs)

    s = sortperm(idxs)
    return [idxs[i] => profs[i] for i in s ]
end

In [ ]:
function load_prof_expected(filename, parampath="profile.toml")
    path = joinpath(modelsdir, filename, parampath)
        
    if isfile(parampath)
        expected = LilGuys.load_profile(parampath)
    else
        error("file not found $path")
    end

    return expected
end

In [ ]:
function compare_profiles(profiles, expected=nothing; 
        sequential=false, legend=true, limits=(-1.5, 0.8, -15, 3), 
        kwargs...
    )
    
    fig = Figure()

	ax = Axis(fig[1,1], xlabel=L"\log\, r / \textrm{kpc}", ylabel =  L"\log\, \rho_\star\; [10^{10} M_\odot / \textrm{kpc}^3]", 
		limits=limits; kwargs...
		)

    plot_kwargs = Dict{Symbol, Any}()

    if sequential
        plot_kwargs[:colorrange] = (1, length(profiles))
    end
    
    for i in eachindex(profiles)
        label, profs = profiles[i]
        prof = profs[end].second
        x = prof.log_r
        y = log10.(prof.rho)
        if sequential
            plot_kwargs[:color] = i
        end

    	lines!(x, y, label=string(label); plot_kwargs...)
        arrows!([log10.(prof.r_break)],[-10], [0], [3], color=COLORS[i])
    end

    if expected !== nothing
    	log_r_pred = LinRange(-2, 2, 1000)
    	ρ_s_pred = calc_ρ.(expected, 10 .^ log_r_pred)

    	lines!(log_r_pred, log10.(ρ_s_pred), label="expected", color="black", linestyle=:dot)
    end

    if legend
    	axislegend(ax, position=:lb)
    end 
    
	fig
end

In [ ]:
function plot_v_t!(profiles; kwargs...)
	sigmas = [p.second.sigma_vx for p in profiles]

    t = [p.second.time for p in profiles] 
	scatterlines!(t * T2GYR, sigmas * V2KMS; kwargs...)
end

In [ ]:
function plot_v_t(profiles; kwargs...)
	fig, ax = FigAxis(
		xlabel = "time / Gyr",
		ylabel = L"\sigma_v / \textrm{km s^{-1}}";
        kwargs...
	)

    plot_v_t!(profiles)

    fig
end
    

In [ ]:
function compare_v_t(profilesi, profs_iso=nothing)
	fig, ax = FigAxis(
		xlabel = "time / Gyr",
		ylabel = L"\sigma_v / \textrm{km s^{-1}}",
	)


    for i in eachindex(profilesi)
        label, profiles = profilesi[i]

        kwargs = Dict{Symbol, Any}()
        kwargs[:label] = label
        kwargs[:color] = COLORS[i]
  
        plot_v_t!(profiles; kwargs...)
    end

    if profs_iso !== nothing
        for i in eachindex(profs_iso)
            label, profiles = profs_iso[i]
    
      
            sigmas = [p.second.sigma_vx for p in profiles]
    
            t = [p.second.time for p in profiles] 
            t = t .- t[end]
            scatterlines!(t * T2GYR, sigmas * V2KMS, color=COLORS[i])
            
        end
    end
    
    hlines!(obs_props["sigma_v"], color=:grey)
    hspan!(obs_props["sigma_v"] - obs_props["sigma_v_err"], 
        obs_props["sigma_v"] + obs_props["sigma_v_err"],
        color = (:grey, 0.1), label="observed")

    LilGuys.Plots.hide_grid!(ax)
    axislegend()
	fig
end

In [ ]:
profs = [
    "compact" => load_profile("sculptor/1e6_V31_r3.2/orbit1/", "exp2d_rs0.10"),
    "smallperi" => load_profile("sculptor/1e6_V31_r3.2/orbit_smallperi", "exp2d_rs0.10"),
    "heavy" => load_profile("sculptor/1e6_V40_r5.9/orbit_mean", "exp2d_rs0.07"),
    "default" => load_profile("sculptor/1e6_V31_r5.9/orbit1", "exp2d_rs0.10"),
    "1e7" => load_profile("sculptor/1e7_V31_r3.2/orbit_mean", "exp2d_rs0.10"),
];

In [ ]:
profs_iso = [
    "compact" => load_isolation("sculptor/1e6_V31_r3.2/orbit1/", "exp2d_rs0.10"),
    "smallperi" => load_isolation("sculptor/1e6_V31_r3.2/orbit_smallperi", "exp2d_rs0.10"),
    "heavy" => load_isolation("sculptor/1e6_V40_r5.9/orbit_mean", "exp2d_rs0.07"),
    "default" => load_isolation("sculptor/1e6_V31_r5.9/orbit1", "exp2d_rs0.10"),
    "1e7" => load_isolation("sculptor/1e7_V31_r3.2/orbit_mean", "exp2d_rs0.10"),
];

In [ ]:
compare_v_t(profs, profs_iso)

In [ ]:
compare_profiles(profs)

In [ ]:
compare_profiles(profs_iso)